<a href="https://colab.research.google.com/github/ysg81/BIG_DATA/blob/main/%EB%AC%B8%EC%84%9C%EB%B2%A1%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import csv
import ast
import copy
import logging

from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [ ]:
# 형태소 분해 된 파일 읽는 코드
 
f = open('/gdrive/My Drive/빅데이터/record3_NLTK.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)

morpheme_separatied_datas = []
morpheme_separatied_data_dic = {}

morpheme_separatied_type = 5

is_content = False
ind = 0
for line in rdr:
    if(is_content):
        morpheme_separatied_datas.append([line[0], line[1], line[2], line[3], ast.literal_eval(line[morpheme_separatied_type])] )
        morpheme_separatied_data_dic[line[0]] = ind
        ind += 1
    is_content = True
 
f.close()

In [ ]:
# 문서벡터 모델을 만드는 코드 

TRAIN_documents = [TaggedDocument(words=text, tags=[tags]) for tags, x1, x2, x3, text in morpheme_separatied_datas]

class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
    def on_epoch_end(self, model):
        self.epoch += 1
        print('epoch {}'.format(self.epoch))
        
vector_size = 256
model = Doc2Vec(TRAIN_documents, vector_size=vector_size, window=3, epochs=80, min_count=0, workers=4, callbacks=[callback()])

In [ ]:
# 만든 모델을 저장하는 코드
model.save('/gdrive/My Drive/빅데이터/summary_model.doc2vec')

In [ ]:
# 저장한 모델을 불러오는 코드
model = Doc2Vec.load('/gdrive/My Drive/빅데이터/summary_model.doc2vec')

In [ ]:
# 만든 모델로 트레이닝 데이터를 다시 가공하는 코드

doc_vectorlized_datas = copy.deepcopy(morpheme_separatied_datas)

for i in range(0,len(doc_vectorlized_datas)):
    if(i%100 == 0): print(i)
    trained_doc_vec = model.docvecs[doc_vectorlized_datas[i][0]]
    inferred_doc_vec = model.infer_vector(doc_vectorlized_datas[i][4])
    doc_vectorlized_datas[i].append(inferred_doc_vec)

In [ ]:
# 가공된 트레이닝 데이터를 csv 파일로 저장하는 코드

with open('listfile.csv', 'w', newline='') as f: 
    writer = csv.writer(f)
    for i in range(len(doc_vectorlized_datas)):
        if(i%100 == 0): print(i)
        writer.writerow(doc_vectorlized_datas)


In [ ]:
# 형태소분해 된 문장 중 가장 비슷한 문장 id를 찾는 코드


def finding_similar(new_document, count = 1):

    inferred_v = model_loaded.infer_vector(new_document)
    model.docvecs.init_sims()
    return model_loaded.docvecs.most_similar([inferred_v], topn=count)

# [[문서id, 유사도]...count개]

